In [ ]:
# 🐺 WOLF PACK RESEARCH VALIDATION
# Testing Fenrir's Claims with ACTUAL DATA
# January 10, 2026 - Brokkr Running the Numbers

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 200)

print("🐺 BROKKR RESEARCH LAB")
print("=" * 60)
print("Let's test Fenrir's claims with REAL data")
print("=" * 60)


: 

In [ ]:
# TICKER UNIVERSE
TICKERS = {
    'nuclear': ['DNN', 'UEC', 'UUUU', 'SMR', 'CCJ', 'URG'],
    'ai_infra': ['CIFR', 'WULF', 'CLSK', 'BTBT', 'APLD', 'IREN'],
}

ALL_TICKERS = [t for sector in TICKERS.values() for t in sector]

print(f"Testing {len(ALL_TICKERS)} tickers: {', '.join(ALL_TICKERS)}")


In [ ]:
# LOAD DATA (180 days)
print("\n📊 Loading data...")

data = {}
for ticker in ALL_TICKERS:
    try:
        df = yf.download(ticker, period='180d', interval='1d', progress=False)
        if not df.empty:
            # Calculate RSI
            delta = df['Close'].diff()
            gain = delta.where(delta > 0, 0).rolling(14).mean()
            loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
            rs = gain / loss
            df['RSI'] = 100 - (100 / (1 + rs))
            
            # Calculate returns
            df['Return'] = df['Close'].pct_change() * 100
            df['Return_5d'] = ((df['Close'] / df['Close'].shift(5)) - 1) * 100
            
            # Volume ratio
            df['Vol_Avg'] = df['Volume'].rolling(20).mean()
            df['Vol_Ratio'] = df['Volume'] / df['Vol_Avg']
            
            # Gap
            df['Gap'] = ((df['Open'] - df['Close'].shift(1)) / df['Close'].shift(1)) * 100
            
            # Day of week
            df['DayOfWeek'] = df.index.dayofweek
            
            # Consecutive days
            df['Green'] = df['Return'] > 0
            df['Consec_Green'] = df['Green'].groupby((~df['Green']).cumsum()).cumsum()
            
            data[ticker] = df
            print(f"  ✓ {ticker}: {len(df)} days, current ${df['Close'].iloc[-1]:.2f}")
    except Exception as e:
        print(f"  ✗ {ticker}: {e}")

print(f"\n✅ Loaded {len(data)} tickers successfully")


In [ ]:
# TEST 1: FENRIR'S CLAIM - "Nuclear is topped out, RSI 73-91"
print("\n" + "="*60)
print("🔬 TEST 1: Current RSI Status")
print("="*60)

current_status = []
for ticker, df in data.items():
    current_rsi = df['RSI'].iloc[-1]
    current_price = df['Close'].iloc[-1]
    ret_5d = df['Return_5d'].iloc[-1]
    
    if current_rsi > 70:
        status = '🔴 OVERBOUGHT'
    elif current_rsi < 30:
        status = '🟢 OVERSOLD'
    elif current_rsi < 40:
        status = '🟡 LOW'
    else:
        status = '⚪ NEUTRAL'
    
    sector = 'Nuclear' if ticker in TICKERS['nuclear'] else 'AI Infra'
    
    current_status.append({
        'Ticker': ticker,
        'Sector': sector,
        'Price': f'${current_price:.2f}',
        'RSI': f'{current_rsi:.1f}',
        'Status': status,
        '5D%': f'{ret_5d:+.1f}%'
    })

status_df = pd.DataFrame(current_status).sort_values('RSI', ascending=False)
print(status_df.to_string(index=False))

# Verdict
nuclear_rsis = [float(x['RSI']) for x in current_status if x['Sector'] == 'Nuclear']
ai_rsis = [float(x['RSI']) for x in current_status if x['Sector'] == 'AI Infra']

print(f"\n📊 VERDICT:")
print(f"   Nuclear avg RSI: {np.mean(nuclear_rsis):.1f}")
print(f"   AI Infra avg RSI: {np.mean(ai_rsis):.1f}")

if np.mean(nuclear_rsis) > 70:
    print(f"   ✅ FENRIR CORRECT: Nuclear is overbought")
else:
    print(f"   ❌ FENRIR WRONG: Nuclear not overbought")


In [ ]:
# TEST 2: FENRIR'S CLAIM - "RSI < 30 is not an edge (~51% win rate)"
print("\n" + "="*60)
print("🔬 TEST 2: RSI < 30 Bounce Pattern")
print("="*60)

rsi_results = []

for ticker, df in data.items():
    # Find RSI < 30 signals
    signals = df[df['RSI'] < 30].copy()
    
    if len(signals) == 0:
        continue
    
    # Calculate forward returns (5 days ahead)
    wins = 0
    total = 0
    returns = []
    
    for idx in signals.index:
        try:
            loc = df.index.get_loc(idx)
            if loc + 5 < len(df):
                future_ret = ((df['Close'].iloc[loc + 5] - df['Close'].iloc[loc]) / df['Close'].iloc[loc]) * 100
                returns.append(future_ret)
                if future_ret > 5:  # Win = >5% gain
                    wins += 1
                total += 1
        except:
            pass
    
    if total > 0:
        win_rate = (wins / total) * 100
        avg_return = np.mean(returns)
        
        rsi_results.append({
            'Ticker': ticker,
            'Events': total,
            'Wins': wins,
            'Win_Rate': f'{win_rate:.0f}%',
            'Avg_Return': f'{avg_return:+.1f}%'
        })

rsi_df = pd.DataFrame(rsi_results)
print(rsi_df.to_string(index=False))

# Calculate aggregate
total_events = rsi_df['Events'].sum()
total_wins = rsi_df['Wins'].sum()
agg_win_rate = (total_wins / total_events) * 100 if total_events > 0 else 0

print(f"\n📊 AGGREGATE:")
print(f"   Total events: {total_events}")
print(f"   Total wins: {total_wins}")
print(f"   Win rate: {agg_win_rate:.1f}%")

if agg_win_rate < 60:
    print(f"   ✅ FENRIR CORRECT: RSI < 30 is NOT an edge ({agg_win_rate:.0f}% < 60%)")
else:
    print(f"   ❌ FENRIR WRONG: RSI < 30 IS an edge ({agg_win_rate:.0f}% > 60%)")


In [ ]:
# TEST 3: FENRIR'S CLAIM - "WULF bounces after -5% drop (64% win rate)"
print("\n" + "="*60)
print("🔬 TEST 3: WULF Drop Bounce Pattern")
print("="*60)

if 'WULF' in data:
    wulf = data['WULF']
    
    # Find days after -5% drop
    big_drops = wulf[wulf['Return'] <= -5.0].copy()
    
    wins = 0
    total = 0
    returns = []
    
    for idx in big_drops.index:
        try:
            loc = wulf.index.get_loc(idx)
            if loc + 1 < len(wulf):
                # Next day return
                next_ret = wulf['Return'].iloc[loc + 1]
                returns.append(next_ret)
                if next_ret > 0:
                    wins += 1
                total += 1
        except:
            pass
    
    if total > 0:
        win_rate = (wins / total) * 100
        avg_return = np.mean(returns)
        
        print(f"Events (WULF -5%+ drops): {total}")
        print(f"Next day green: {wins}/{total}")
        print(f"Win rate: {win_rate:.1f}%")
        print(f"Avg next day: {avg_return:+.2f}%")
        
        if 60 <= win_rate <= 70:
            print(f"\n✅ FENRIR CORRECT: WULF bounces ~{win_rate:.0f}%")
        else:
            print(f"\n❌ FENRIR WRONG: Claimed 64%, actual {win_rate:.0f}%")
else:
    print("WULF data not available")


In [ ]:
# TEST 4: FENRIR'S CLAIM - "DNN gaps up 3%+ continue (67% win rate)"
print("\n" + "="*60)
print("🔬 TEST 4: DNN Gap Continuation Pattern")
print("="*60)

if 'DNN' in data:
    dnn = data['DNN']
    
    # Find gap up days (3%+)
    gap_ups = dnn[dnn['Gap'] >= 3.0].copy()
    
    continued = 0
    filled = 0
    total = 0
    
    for idx in gap_ups.index:
        try:
            # Did it close above open (continue) or below (fill)?
            close_price = dnn.loc[idx, 'Close']
            open_price = dnn.loc[idx, 'Open']
            
            if close_price > open_price:
                continued += 1
            else:
                filled += 1
            total += 1
        except:
            pass
    
    if total > 0:
        continue_rate = (continued / total) * 100
        fill_rate = (filled / total) * 100
        
        print(f"Gap up 3%+ events: {total}")
        print(f"Continued (close > open): {continued}/{total} = {continue_rate:.0f}%")
        print(f"Filled (close < open): {filled}/{total} = {fill_rate:.0f}%")
        
        if 60 <= continue_rate <= 75:
            print(f"\n✅ FENRIR CORRECT: DNN gaps continue ~{continue_rate:.0f}%")
        else:
            print(f"\n❌ FENRIR WRONG: Claimed 67%, actual {continue_rate:.0f}%")
    else:
        print("No gap up events in period")
else:
    print("DNN data not available")


In [ ]:
# TEST 5: FENRIR'S CLAIM - "Wednesday is best day (55% win rate)"
print("\n" + "="*60)
print("🔬 TEST 5: Day of Week Performance")
print("="*60)

dow_results = []

for day in range(5):  # Mon=0, Fri=4
    day_name = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'][day]
    
    all_returns = []
    wins = 0
    total = 0
    
    for ticker, df in data.items():
        day_data = df[df['DayOfWeek'] == day]['Return'].dropna()
        all_returns.extend(day_data.tolist())
        wins += (day_data > 0).sum()
        total += len(day_data)
    
    if total > 0:
        avg_return = np.mean(all_returns)
        win_rate = (wins / total) * 100
        
        dow_results.append({
            'Day': day_name,
            'Count': total,
            'Wins': wins,
            'Win_Rate': f'{win_rate:.1f}%',
            'Avg_Return': f'{avg_return:+.2f}%'
        })

dow_df = pd.DataFrame(dow_results)
print(dow_df.to_string(index=False))

# Find best day
best_day = dow_df.iloc[dow_df['Avg_Return'].apply(lambda x: float(x.strip('%+'))).argmax()]
print(f"\n📊 Best day: {best_day['Day']} ({best_day['Win_Rate']} win rate, {best_day['Avg_Return']} avg)")

if 'Wednesday' in best_day['Day']:
    print(f"✅ FENRIR CORRECT: Wednesday is best day")
else:
    print(f"❌ FENRIR WRONG: Best day is {best_day['Day']}, not Wednesday")


In [ ]:
# TEST 6: FENRIR'S CLAIM - "Leader/laggard same day only, no next-day edge"
print("\n" + "="*60)
print("🔬 TEST 6: CCJ Leader → DNN/UEC Laggard")
print("="*60)

if 'CCJ' in data and 'DNN' in data and 'UEC' in data:
    ccj = data['CCJ']
    
    # Find CCJ big up days (3%+)
    ccj_big_days = ccj[ccj['Return'] >= 3.0].index
    
    print(f"CCJ big up days (3%+): {len(ccj_big_days)}")
    
    for laggard_name in ['DNN', 'UEC']:
        laggard = data[laggard_name]
        
        same_day_green = 0
        next_day_green = 0
        count = 0
        
        for date in ccj_big_days:
            if date not in laggard.index:
                continue
            
            # Same day
            same_ret = laggard.loc[date, 'Return']
            if same_ret > 0:
                same_day_green += 1
            
            # Next day
            try:
                loc = laggard.index.get_loc(date)
                if loc + 1 < len(laggard):
                    next_ret = laggard['Return'].iloc[loc + 1]
                    if next_ret > 0:
                        next_day_green += 1
            except:
                pass
            
            count += 1
        
        if count > 0:
            same_pct = (same_day_green / count) * 100
            next_pct = (next_day_green / count) * 100
            
            print(f"\n{laggard_name}:")
            print(f"  Same day green: {same_day_green}/{count} = {same_pct:.0f}%")
            print(f"  Next day green: {next_day_green}/{count} = {next_pct:.0f}%")
            
            if same_pct > 90 and 45 <= next_pct <= 55:
                print(f"  ✅ FENRIR CORRECT: Same day correlation, next day random")
            else:
                print(f"  ⚠️ Needs review: Same {same_pct:.0f}%, Next {next_pct:.0f}%")
else:
    print("Missing ticker data")


In [ ]:
# FINAL SUMMARY
print("\n" + "="*60)
print("🐺 FINAL VERDICT: FENRIR'S RESEARCH")
print("="*60)
print("\nTests completed. Review results above.")
print("\nNext steps:")
print("1. If patterns validated → Code into scanners")
print("2. If patterns failed → Discard them")
print("3. Paper trade validated patterns for 2 weeks")
print("4. Only trade with real money after paper success")
print("\n🐺 LLHR. Data > Claims. AWOOOO.")
